<a href="https://colab.research.google.com/github/chunabayashi0216/SIGNATE_sinfuzen/blob/main/sinfuzen_lightBGM_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 心不全　LightGBM (Python Scikit-learn interface)　その３

# 交差検証(Cross Validation)

# パラメーター調整（グリッドサーチ）

# 'platelets' 'serum_creatinine' を整数変換

In [ ]:
'''
SIGNATE　提出  04/12/2021 0.850  StratifiedKFold, n_splits=5,
                                param={'learning_rate': 0.1, 'max_depth': 4, 'num_leaves': 20, 'reg_alpha': 1, 'reg_lambda': 0}

                                'platelets'　 float -> int
                                'serum_creatinine' 1000倍してfloat -> int
'''

"\nSIGNATE\u3000提出 04/10/2021 0.844\n              04/12/2021 0.845  StratifiedKFold, n_splits=5,\n                                param={'learning_rate': 0.1, 'max_depth': 4, 'num_leaves': 20, 'reg_alpha': 1, 'reg_lambda': 0}\n"

In [ ]:
%pip install japanize-matplotlib

## ライブラリのインポート
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')
import lightgbm as lgb #LightGBM
from sklearn import datasets
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import accuracy_score # モデル評価用(正答率)
from sklearn.metrics import log_loss # モデル評価用(logloss)
from sklearn.metrics import roc_auc_score # モデル評価用(auc)

Note: you may need to restart the kernel to use updated packages.


【データの読み込み、トレーニングデータ・テストデータの分割】

In [ ]:
%cd C:\Users\m_chunabayashi\SIGNATE\心不全予測\

# Load in the train and test datasets
df_train = pd.read_csv('train.csv') #学習データ
df_test = pd.read_csv('test.csv')   #テストデータ

# データの確認
print(df_train.shape) # 学習データサイズの確認(データ数,特徴量数)
print(df_test.shape)  # テストデータサイズの確認(データ数,特徴量数)


C:\Users\m_chunabayashi\SIGNATE\心不全予測
(1000, 14)
(1000, 13)


特徴量　データ加工　float -> int

In [ ]:
# 'serum_creatinine' float -> int
df_train['platelets'] = df_train['platelets'].astype(int)
df_test['platelets'] = df_test['platelets'].astype(int)

# 'serum_sodium' float -> int
df_train['serum_creatinine'] = df_train['serum_creatinine'] * 1000
df_train['serum_creatinine'] = df_train['serum_creatinine'].astype(int)

df_test['serum_creatinine'] = df_test['serum_creatinine'] * 1000
df_test['serum_creatinine'] = df_test['serum_creatinine'].astype(int)



In [ ]:
# 説明変数,目的変数
X = df_train.drop(['id', 'target'],axis=1).values # 説明変数(index 'target'以外の特徴量)
y = df_train['target'].values                        # 目的変数('target')

df_test = df_test.drop(['id'],axis=1)    # 提出データ

print(X.shape) # 学習データサイズの確認(データ数,特徴量数)
print(y.shape)  # テストデータサイズの確認(データ数,特徴量数)
print(df_test.shape)

(1000, 12)
(1000,)
(1000, 12)


【心不全発症の比率】

In [ ]:
n_target0, n_target1 = len(df_train[df_train['target'] == 0]), len(df_train[df_train['target'] == 1])
n_all = n_target0 + n_target1
print('Outcome = 0 の割合 :',n_target0/n_all) # target0 健常者)の割合
print('Outcome = 1 の割合 :',n_target1/n_all) # target1　心不全発症の割合

Outcome = 0 の割合 : 0.802
Outcome = 1 の割合 : 0.198


Scikit-learn interfaceを使用する

In [ ]:
# トレーニングデータをtrainとvalidに分割
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.20, random_state=0)

# LightGBMの分類器をインスタンス化
gbm = lgb.LGBMClassifier(objective='binary')  # , importance_type='gain'

# trainとvalidを指定し学習
gbm.fit(train_x, train_y,
        eval_set = [(train_x, train_y), (valid_x, valid_y)],
        early_stopping_rounds=20,
        verbose=-1)

# valid_xについて推論
oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
print('score', round(accuracy_score(valid_y, oof)*100,2));  # validのscore

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[43]	training's binary_logloss: 0.105826	valid_1's binary_logloss: 0.381696
score 83.0


In [ ]:
gbm.best_iteration_

43

パラメータの調整

In [ ]:
gbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': 'binary',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [ ]:
# GridSearchCVをimport
from sklearn.model_selection import GridSearchCV

gbm = lgb.LGBMClassifier(objective='binary')

# 試行するパラメータを羅列する
#params = {
#    'num_leaves': [ 10, 20, 30, 40],
#    'reg_alpha': [0, 1, 10, 100],
#    'reg_lambda': [0, 1, 10, 100],
#}

params = {
    'max_depth':[2, 3, 4, 5, 6, 7, 8],
    'learning_rate':[ 0.01, 0.1, 1],
    'num_leaves': [ 10, 20, 30, 40],
    'reg_alpha': [0, 1, 10, 100],
    'reg_lambda': [0, 1, 10, 100],
}

grid_search = GridSearchCV(
                           gbm,  # 分類器を渡す
                           param_grid=params,  # 試行してほしいパラメータを渡す
                           cv=5,  # 3分割交差検証でスコアを確認
                          )

grid_search.fit(X, y)  # データを渡す

print(grid_search.best_score_)  # ベストスコアを表示
print(grid_search.best_params_)  # ベストスコアのパラメータを表示

0.8620000000000001
{'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 20, 'reg_alpha': 0, 'reg_lambda': 0}


① KFold {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 20, 'reg_alpha': 0, 'reg_lambda': 0}

In [ ]:
# 訓練

from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=0)

# スコアとモデルを格納するリスト
score_list = []
logloss_list = []
auc_list = []
models = []

test_pred = np.zeros((len(df_test), 10))

for fold_, (train_index, valid_index) in enumerate(kf.split(X, y)):
    train_x = X[train_index]
    valid_x = X[valid_index]
    train_y = y[train_index]
    valid_y = y[valid_index]

    print(f'fold{fold_ + 1} start')

    gbm = lgb.LGBMClassifier(objective='binary',
                             num_iterations=1000,
                             learning_rate= 0.1,
                             max_depth= 7 ,
                             num_leaves=20,
                             reg_alpha=0,
                             reg_lambda=0)  # パラメータを指定
    gbm.fit(train_x, train_y,
            eval_set = [(train_x, train_y), (valid_x, valid_y)],
            early_stopping_rounds=20,
            verbose= -1)

    # テストデータの予測 (クラス1の予測確率(クラス1である確率)を返す)
    y_pred_prob = gbm.predict_proba(valid_x, num_iteration=gbm.best_iteration_)
    print('gbm.best_iteration_: ', gbm.best_iteration_)

    oof = np.where(y_pred_prob[:, 1] > 0.5, 1, 0) # 0.5より大きい場合1 ,そうでない場合0を返す

    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    logloss_list.append(log_loss(valid_y,y_pred_prob[:, 1]))  # logloss
    auc_list.append(roc_auc_score(valid_y,y_pred_prob[:, 1])) # AUC
    models.append(gbm)                                 # 学習が終わったモデルをリストに入れておく

    print(f'fold{fold_ + 1} end\n' )

# モデル評価結果
print(score_list, '平均score', np.mean(score_list))
print(logloss_list, '平均logloss', np.mean(logloss_list))
print(auc_list, '平均auc', np.mean(auc_list))

fold1 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[53]	training's binary_logloss: 0.127639	valid_1's binary_logloss: 0.355215
gbm.best_iteration_:  53
fold1 end

fold2 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[31]	training's binary_logloss: 0.200246	valid_1's binary_logloss: 0.323377
gbm.best_iteration_:  31
fold2 end

fold3 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[66]	training's binary_logloss: 0.110748	valid_1's binary_logloss: 0.296995
gbm.best_iteration_:  66
fold3 end

fold4 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[51]	training's binary_logloss: 0.130775	valid_1's binary_logloss: 0.386858
gbm.best_iteration_:  51
fold4 end

fold5 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[43]	trainin

In [ ]:
# 提出
# テストデータの予測を格納する、1000行10列のnumpy行列を作成
test_pred = np.zeros((len(df_test), 10))

for i, gbm in enumerate(models):
    test_pred[:, i] = gbm.predict(df_test) # testを予測

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)

df_submit = pd.read_csv('sample_submission.csv', header=None, names=['index', 'target'])
df_submit['target'] = pred

df_submit.to_csv('submit_0410.csv', header=False, index=False )




#pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
#sample_submission['Survived'] = pred
#sample_submission.to_csv('glid_search.csv', index=False)

②StratifiedKFold

In [ ]:
from sklearn.model_selection import StratifiedKFold
Sf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)


# スコアとモデルを格納するリスト
score_list = []
logloss_list = []
auc_list = []
models = []

test_pred = np.zeros((len(df_test), 10))

for fold_, (train_index, valid_index) in enumerate(Sf.split(X, y)):
    train_x = X[train_index]
    valid_x = X[valid_index]
    train_y = y[train_index]
    valid_y = y[valid_index]

    print(f'fold{fold_ + 1} start')

    gbm = lgb.LGBMClassifier(objective='binary',
                             num_iterations=1000,
                             learning_rate= 0.1,
                             max_depth= 7 ,
                             num_leaves=20,
                             reg_alpha=0,
                             reg_lambda=0)  # パラメータを指定

    gbm.fit(train_x, train_y,
            eval_set = [(train_x, train_y), (valid_x, valid_y)],
            early_stopping_rounds=20,
            verbose= -1)

    # テストデータの予測 (クラス1の予測確率(クラス1である確率)を返す)
    y_pred_prob = gbm.predict_proba(valid_x, num_iteration=gbm.best_iteration_)
    print('gbm.best_iteration_: ', gbm.best_iteration_)

    oof = np.where(y_pred_prob[:, 1] > 0.5, 1, 0) # 0.5より大きい場合1 ,そうでない場合0を返す

    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    logloss_list.append(log_loss(valid_y,y_pred_prob[:, 1]))  # logloss
    auc_list.append(roc_auc_score(valid_y,y_pred_prob[:, 1])) # AUC
    models.append(gbm)                                 # 学習が終わったモデルをリストに入れておく

    print(f'fold{fold_ + 1} end\n' )

# モデル評価結果
print(score_list, '平均score', np.mean(score_list))
print(logloss_list, '平均logloss', np.mean(logloss_list))
print(auc_list, '平均auc', np.mean(auc_list))

fold1 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[56]	training's binary_logloss: 0.134537	valid_1's binary_logloss: 0.320139
gbm.best_iteration_:  56
fold1 end

fold2 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[41]	training's binary_logloss: 0.160125	valid_1's binary_logloss: 0.356771
gbm.best_iteration_:  41
fold2 end

fold3 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[86]	training's binary_logloss: 0.0930371	valid_1's binary_logloss: 0.32493
gbm.best_iteration_:  86
fold3 end

fold4 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[52]	training's binary_logloss: 0.141515	valid_1's binary_logloss: 0.28678
gbm.best_iteration_:  52
fold4 end

fold5 start
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[42]	training

In [ ]:
# 提出
# テストデータの予測を格納する、1000行10列のnumpy行列を作成
test_pred = np.zeros((len(df_test), 5))

for i, gbm in enumerate(models):
    test_pred[:, i] = gbm.predict(df_test) # testを予測

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)

df_submit = pd.read_csv('sample_submission.csv', header=None, names=['index', 'target'])
df_submit['target'] = pred

df_submit.to_csv('submit_0412_04.csv', header=False, index=False )
